In [45]:
from pynqOverlay import *
from pynqOverlayDriver import *
#viene richiamata la classe per la gestione del filtro e dell'hdmi
po = pynqOverlay()

In [46]:
# configurazione monitor: 640*480 @ 60Hz

Mode = VideoMode(640,480,24) 
hdmi_out = po.ol.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR) 
hdmi_out.start()

In [47]:
#configurazione camera (input)
frame_in_w = 640
frame_in_h = 480

In [48]:
#inizializzazione camera con OpenCV
import cv2

videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);
print("cattura device is open: " + str(videoIn.isOpened()))

cattura device is open: True


In [49]:
# manda linput della webcam all'output dell'HDMI
import numpy as np

ret, frame_vga = videoIn.read()

if (ret):
    outframe = hdmi_out.newframe()      
    outframe[:] = frame_vga
    hdmi_out.writeframe(outframe)
else:
    raise RuntimeError("Error while reading from camera.")

In [50]:
#Filtraggio dei frame
num_frames = 50
readError = 0


for i in range (num_frames):   
    # legge la prossima immagine e applica il filtro
    ret, frame_vga = videoIn.read()
    if (ret):
        outframe = hdmi_out.newframe()               
        po.Sobel(frame_vga,-1,1,0,dst=outframe)        
        hdmi_out.writeframe(outframe)
    else:
        readError += 1

print("Number of read errors: " + str(readError))

Number of read errors: 0


In [51]:
#chiusura dei canali

videoIn.release()
hdmi_out.stop()